####  Finding optimal hyperparameters for one of the algorithms (LightGBM in this case)

In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

import tensorflow as tf
import keras
from keras import layers

#### Load the data

In [7]:
df = pd.read_csv("housing_before_optimization.csv")
df.head()

,housing_median_age,total_rooms,population,median_income,median_house_value,distance_to_nearest_city,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY
0,41.0,880.0,322.0,8.3252,452600.0,20.33,0,0,0,1
1,21.0,7099.0,2401.0,8.3014,358500.0,19.91,0,0,0,1
2,52.0,1467.0,496.0,7.2574,352100.0,17.84,0,0,0,1
3,52.0,1274.0,558.0,5.6431,341300.0,17.06,0,0,0,1
4,52.0,1627.0,565.0,3.8462,342200.0,17.06,0,0,0,1


####  Same X/y + train/test split

In [8]:
categorical_variables = [ 
    "ocean_proximity_<1H OCEAN",
    "ocean_proximity_INLAND",
    "ocean_proximity_ISLAND",
    "ocean_proximity_NEAR BAY"]

# continuous variables also into a list
continuous_variables = [ 
    "housing_median_age",
    "total_rooms",
    "population",
    "median_income",
    "distance_to_nearest_city",
    "median_house_value"]

# the usual X/y -split
X = df.drop("median_house_value", axis=1)
y = df['median_house_value']

# usual train/test -split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# NOTE! SCALING => some of the algorithms require this
scaler = StandardScaler()

# create separate versions for the scaled data
# because we need both unscaled and scaled versions later
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

####  In this example, use RandomizedSearchCV to search better hyperparameters for our algorithm -  CatBoost

#### This example was taken from AI

In [9]:
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Define parameter distributions for CatBoost
param_dist = {
    'depth': randint(4, 10),                 # tree depth
    'learning_rate': uniform(0.03, 0.15),    # learning rate range
    'iterations': randint(300, 1200),        # number of trees
    'l2_leaf_reg': uniform(1, 9),             # L2 regularization
}

# Setup RandomizedSearchCV
random_search_cat = RandomizedSearchCV(
    estimator=CatBoostRegressor(
        loss_function='RMSE',
        verbose=0,
        random_seed=42
    ),
    param_distributions=param_dist,
    n_iter=100,          # CatBoost is slower → 100 is realistic
    cv=3,
    n_jobs=-1,
    verbose=1,
    scoring='neg_mean_squared_error'
)

# Fit search
random_search_cat.fit(X_train, y_train)

# Best parameters
print("Best parameters:", random_search_cat.best_params_)
print("Best CV score (neg MSE):", random_search_cat.best_score_)


Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters: {'depth': 8, 'iterations': 944, 'l2_leaf_reg': np.float64(5.08618378675995), 'learning_rate': np.float64(0.05250387504557257)}
Best CV score (neg MSE): -2547064787.5312743
